# The Battle of Neighborhoods Toronto

### Description on the problem

I have read and heard from friends about advantages of migration to Canada and especially to the city of Toronto, My status as family father and my business as an independant consultant (banks and finance) involves a lot of conditions before taking a big move like moving to a new address. More generally this analysis will benefit to many famillies having the same objectives.

#### Objective
My objective is to evaluate the the opportunity of living in Toronto neighbourhoods based on 4 Criterias
* Proximity to green spots. (family confort)
* Proximity to banks. (workplaces)
* Proximity to beach. (clean air)

#### target audience
This analysis will be useful for families considering a move or a migration to the city of Toronto. The choosen criterias can be changed to adapt to specific needs.

### Data collection and usage
Data will be mainly prepared as follows :
* Boroughs and neighborhouds names will be collected by scrapping wikipedia ( Toronto's addresses page in particular).
* Geolocalisation will be done using  geocoder library and opencage API
* Venues collection will be done using foursquare API.

Once done the data will be grouped by venue's type concentration for each neighborhood. Finally the neighborhoods will be ranked depending on best scores (based on target venue's types concentration). For instance Victoria park neighborhood is a perfect candidate for beeing surrounded by parks, near to important economic areas and not too far from the beach.

### Methodology
We are aiming for a cluster of target neignborhoods that share nearby features by using unsupervised classification (using KMeans). Then we will score each cluster with a weighted map of features (venue types) that will favor the closest cluster to our preference.

the parameters for this study are :
- The weighted list of venue types that are of interest for a given case. (in my case proximity to work and quality of life)
- The number of clusters to group neighbourhoods (More clusters means that we are more selective and want reduced choices)


### install useful modules

In [1]:
# beautiful soup for web scrapping
!pip install beautifulsoup4
# geocoder for geolocalisation
!pip install geocoder
# folium for map rendering
!pip install folium

     |████████████████████████████████| 102kB 2.5MB/s 


### import useful libraries

In [2]:
import numpy as np
import pandas as pd
import requests
import re
import os
from bs4 import BeautifulSoup
import geocoder
from getpass import getpass
import folium

### start scrapping the wikipedia page for neighborhoods of Toronto

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url, allow_redirects=True)
soup = BeautifulSoup(page.content, 'html.parser')

In [4]:
postalcodes_tab = soup.find('table',class_='wikitable')

neighb_df = pd.DataFrame(columns=['PostalCode','Borough','Neighborhood'])
i=0

for neighborhood_tr in postalcodes_tab.find_all('tr'):
  if (len(neighborhood_tr.find_all('td')) == 3) :
    neighb_row = [td.text.rstrip() for td in neighborhood_tr.find_all('td')]
    if (neighb_row[1] != 'Not assigned') :
      neighb_df.loc[i] = neighb_row
      i+=1

neighb_df.head(10)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [5]:
neighb_df.shape


(103, 3)

In [6]:
#opencage_api_key = getpass("what is your opencage api key : ")

opencage_api_key = "load_me_from_drive"

with open('drive/My Drive/Colab Notebooks/keys/opencage') as f:
    opencage_api_key = f.readline()
    f.close

#print (geocoder.opencage('North York, Victoria Village, CA', key=openkage_api_key).latlng)
#print (geocoder.osm('North York, Victoria Village, CA').latlng)

### add geolocalisation data to the frame

In [7]:
neighb_df['address'] = neighb_df['Neighborhood'] + ', ' + neighb_df['Borough']+', Toronto, Canada'
#
neighb_df['lat'] = neighb_df['PostalCode']
neighb_df['lng'] = neighb_df['PostalCode']

#neighb_df_tst = neighb_df.head(3)

#neighb_df_tst['coordinates']=neighb_df_tst['adress'].apply(geocoder.osm).apply(lambda x: x.latlng if x != None else None)

for index, row in neighb_df.iterrows():
  latlng = geocoder.opencage(row['address'], key=opencage_api_key).latlng
  #print (row['adress'], geocoder.opencage(repr(row['adress']), key=opencage_api_key).latlng)
  if (latlng is not None) :
    row['lat'], row['lng'] = latlng[0], latlng[1]

neighb_df
#neighb_df['lat'] = geocoder.osm(neighb_df['adress']).lat
#geocoder.osm('M3A, Parkwoods, North York, CA').latlng

,PostalCode,Borough,Neighborhood,address,lat,lng
0,M3A,North York,Parkwoods,"Parkwoods, North York, Toronto, Canada",43.7611,-79.3241
1,M4A,North York,Victoria Village,"Victoria Village, North York, Toronto, Canada",43.7327,-79.3112
2,M5A,Downtown Toronto,"Regent Park, Harbourfront","Regent Park, Harbourfront, Downtown Toronto, T...",43.7001,-79.4163
3,M6A,North York,"Lawrence Manor, Lawrence Heights","Lawrence Manor, Lawrence Heights, North York, ...",43.7001,-79.4163
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government","Queen's Park, Ontario Provincial Government, D...",43.7001,-79.4163
...,...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North","The Kingsway, Montgomery Road, Old Mill North,...",43.7001,-79.4163
99,M4Y,Downtown Toronto,Church and Wellesley,"Church and Wellesley, Downtown Toronto, Toront...",43.6615,-79.3829
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...","Business reply mail Processing Centre, South C...",45.7236,7.4575
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...","Old Mill South, King's Mill Park, Sunnylea, Hu...",43.7001,-79.4163


In [8]:
address = 'Toronto, CA'

toronto_latlng = geocoder.opencage(address, key=opencage_api_key).latlng

print('The geograpical coordinate of Toronto are {}, {}.'.format(toronto_latlng[0], toronto_latlng[1]))

The geograpical coordinate of Toronto are 33.5731279, -117.7355315.


In [9]:
condition = neighb_df['Borough'].str.contains('Toronto')

neighb_toronto_df = neighb_df[condition]

In [10]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[toronto_latlng[0], toronto_latlng[1]], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighb_toronto_df['lat'], neighb_toronto_df['lng'], neighb_toronto_df['Borough'], neighb_toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Clustering by categories of trending venues

In [11]:
# test the response of one call
CLIENT_ID = 'LETUVSLX3N1JYS23O4KJIJTAMSE2K1WYBCFTZZC52TJ5U5XC' # your Foursquare ID
CLIENT_SECRET = 'HEJX3D2KAC3UN5EOE1PHAONNFQVVRG4KECXPWFODDHXFW2UG' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, 43.7327,	-79.3112, 500, 10)
# make the GET request
results = requests.get(url).json()

#results

{'meta': {'code': 200, 'requestId': '5f4d003e034c6c36a105d9d7'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-550df684498ea2dd2c87bb5a-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/thai_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d149941735',
         'name': 'Thai Restaurant',
         'pluralName': 'Thai Restaurants',
         'primary': True,
         'shortName': 'Thai'}],
       'id': '550df684498ea2dd2c87bb5a',
       'location': {'address': '1744  Victoria Park',
        'cc': 'CA',
        'city': 'North York',
        'country': 'Canada',
        'crossStreet': 'Surrey Ave',
        'distance': 482,
        'formattedAddress': ['1744  Victoria Park (Surrey Ave)',
         'North York ON M1R 1R4',
         'Canada'],
        'labeledLa

In [12]:
def getExploreVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(name, lat, lng, v['venue']['name'], v['venue']['location']['lat'], v['venue']['location']['lng'], v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    return(nearby_venues)

toronto_venues = getExploreVenues(names=neighb_toronto_df['Neighborhood'], latitudes=neighb_toronto_df['lat'], longitudes=neighb_toronto_df['lng'] ) 

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


In [13]:
toronto_venues['Venue Category'].unique()

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues['Venue Category'].unique())

In [14]:
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']
#for col in toronto_onehot.columns :
#  print(col)

In [73]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean()

In [74]:
toronto_grouped

,African Restaurant,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bank,Bar,Beach,Beer Bar,Beer Store,Bookstore,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Café,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Diner,Distribution Center,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Falafel Restaurant,Farmers Market,...,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Mountain,Movie Theater,Museum,Music Venue,Nail Salon,New American Restaurant,North Indian Restaurant,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Sandwich Place,Shoe Store,Shopping Mall,Skating Rink,Supermarket,Sushi Restaurant,Tailor Shop,Tanning Salon,Tattoo Parlor,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
Neighborhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Berczy Park,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.071429,0.071429,0.000000,0.071429,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909
"Garden District, Ryerson",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.071429,0.071429,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,...,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000
"Queen's Park, Ontario Provincial Government",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.071429,0.000000,0.071429,0.071429,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.0

In [75]:
interesting_venue_categories = {'Bank':5,'Beach':2,'Garden':2,'Gym':4,'Museum':4,'Supermarket':2,'Pharmacy':3,'Park':2,'Fountain':2} #this can be customized for other needs

In [76]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 4

#toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([2, 0, 2, 2, 3, 1, 2], dtype=int32)

In [77]:
#toronto_grouped['label'] = kmeans.labels_
#toronto_grouped.head()

In [79]:
toronto_grouped_label=toronto_grouped.groupby('label')
label_sum = None
for interesting_category, coef in interesting_venue_categories.items() :
  if label_sum is None :
    label_sum = toronto_grouped_label[interesting_category].sum() * coef
  else :
    label_sum += toronto_grouped_label[interesting_category].sum() * coef
label_sum

label
0    0.181818
1    0.714286
2    0.500000
3    0.500000
Name: Bank, dtype: float64

### Results

In the results we have a clear winner

In [80]:
best_cluster_condition = toronto_grouped['label'] == 2
best_neighbourhoods = best_cluster_condition[best_cluster_condition]

best_neighbourhoods

Neighborhood
Berczy Park                                    True
Garden District, Ryerson                       True
Queen's Park, Ontario Provincial Government    True
The Beaches                                    True
Name: label, dtype: bool

The winners are :

 * Berczy Park
 * Garden District, Ryerson
 * Queen's Park, Ontario Provincial Government
 * The Beaches                               

### Discussion

Applying bias (weighted map) can be done before segmentation but it would affect the unsupervised clustering, we choose to apply it after clustering to affect only scoring. 

Normally we should add cost of life parameters but it's undirectly included in the type of venues (the more popular the less costly).

### Conclusion
We have done a parametrized study to compare neughboorhoods of the city of Toronto, we focused on a specific case "Fitness for Migration given family preferences". We naturally got a short list of condidates, next step will be manual and will take into consideration non formal and human reviewed criteria. 